## Web Scraping 

In [ ]:
# Installing required libraries
!pip install snowflake-connector-python

In [3]:
# Importing required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import snowflake.connector

In [4]:
# Function to scrape book details from a given URL
def scrape_books(url):
    data = []  # List to store book details
    
    # Iterate through pages 1 to 50
    for page in range(1, 51):
        page_url = f"{url}catalogue/page-{page}.html"
        response = requests.get(page_url)
        
        # Check if the response is successful       
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            book_containers = soup.find_all('article', class_='product_pod')
            
            # Extract book details from each book container           
            for book in book_containers:
                # Extract title, price, rating, and availability of the book
                title = book.find('h3').find('a')['title']
                price = book.find('p', class_='price_color').text
                rating = book.find('p', class_='star-rating')['class'][1]
                availability = book.find('p', class_='instock availability').text.strip()
                # Append extracted details to the data list
                data.append({'Title': title, 'Price': price, 'Rating': rating.capitalize(), 'Availability': availability})
        else:
            # Notify if webpage retrieval fails for a particular page
            print(f"Failed to retrieve webpage for page {page}")
    
    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)
    
    # Return the DataFrame containing book details
    return df

In [5]:
# URL of the bookstore website
base_url = 'https://books.toscrape.com/'

# Call the function to scrape books from multiple an dcreate a Dataframe
df = scrape_books(base_url)

## Data Tranformations

In [7]:
# Converting Price from string to integer value
df['Price'] = pd.to_numeric(df['Price'].str.replace('£', ''), errors='coerce') 

In [9]:
# Converting Availability from string to Boolean value
df['Availability'] = df['Availability'].str.strip().map({'In stock': True, 'Out of stock': False})

In [11]:
# Converting Rating from string to numeric value
rating_mapping = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
df['Rating'] = df['Rating'].map(rating_mapping)

In [12]:
df

,Title,Price,Rating,Availability
0,A Light in the Attic,51.77,3,True
1,Tipping the Velvet,53.74,1,True
2,Soumission,50.10,1,True
3,Sharp Objects,47.82,4,True
4,Sapiens: A Brief History of Humankind,54.23,5,True
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,55.53,1,True
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06,4,True
997,A Spy's Devotion (The Regency Spies of London #1),16.97,5,True
998,1st to Die (Women's Murder Club #1),53.98,1,True


## Database Uploading

In [13]:
# Snowflake connection parameters
account = 'vniqwci-jj28244'
user = 'varun'
password = 'Blend360'
warehouse = 'CCW3_WH'
database = 'CCW3_DB'

In [14]:
# Snowflake connection
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account
)

In [15]:
# Snowflake cursor
cur = conn.cursor()

In [22]:
cur.execute(f"DROP WAREHOUSE IF EXISTS {warehouse}")
cur.execute(f"DROP DATABASE IF EXISTS {database}")

In [23]:
# Create warehouse if it doesn't exist
cur.execute(f"CREATE WAREHOUSE IF NOT EXISTS {warehouse}")
# Use the warehouse
cur.execute(f"USE WAREHOUSE {warehouse}")

In [24]:
# Create database if it doesn't exist
cur.execute(f"CREATE DATABASE IF NOT EXISTS {database}")

# Use the database
cur.execute(f"USE DATABASE {database}")

In [25]:
# Create table in Snowflake
table_name = 'books_table'
cur.execute(f"CREATE TABLE IF NOT EXISTS {table_name} (Title STRING, Price NUMBER, Rating NUMBER, Availability BOOLEAN)")

In [26]:
# Insert data from Dataframe into Database 
insert_query = f"INSERT INTO {table_name} (Title, Price, Rating, Availability) VALUES (%s, %s, %s, %s)"
for index, row in df.iterrows():
    cur.execute(insert_query, (row['Title'], row['Price'], row['Rating'], row['Availability']))

In [ ]:
# Commit changes
cur.execute("COMMIT")

In [ ]:
# Close cursor and connection
cur.close()
conn.close()